In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

C:\Python64\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv("../data/Prostate Cancer.txt") 
names = ['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']
df_train = df[df.train == 'T']
df_valid = df[df.train == 'F']
X, y = df_train[names].values, df_train[['lpsa']].values
X_valid, y_valid = df_valid[names].values, df_valid[['lpsa']].values
df[names].corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652
lweight,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354
age,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112
lbph,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460
svi,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648
lcp,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528
gleason,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905
pgg45,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000


In [3]:
y_mean = np.mean(y)
print('Baseline RMSE: ', np.mean((y_valid-y_mean)**2)**0.5)

Baseline RMSE:  1.0279753051802276


In [4]:
# I don't think that scaling based on the whole data set is a good idea
# but without it we get different (compared to the book) coefficients from linear regression 
scaler = StandardScaler()
scaler.fit(np.vstack([X, X_valid]))
X = scaler.transform(X)
X_valid = scaler.transform(X_valid)

In [6]:
#reg = LinearRegression().fit(X, y)
#print(reg.intercept_, reg.coef_[0])
ls = sm.OLS(y, sm.add_constant(X)).fit()
print(ls.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                     16.47
Date:                Fri, 25 Jan 2019   Prob (F-statistic):           2.04e-12
Time:                        12:29:22   Log-Likelihood:                -67.505
No. Observations:                  67   AIC:                             153.0
Df Residuals:                      58   BIC:                             172.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4649      0.089     27.598      0.0